<a href="https://colab.research.google.com/github/tadinve/rsa_on_aws/blob/main/notebooks/00_TF_Cadabra_Infra_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Do this only Once
- Create an account in AWS
- Create an IAM User and downlod credentials file (you will need access and secret key)



#Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

http://checkip.dyndns.org

In [ ]:
#these need to be changed
%env TF_VAR_name_initials=
%env TF_VAR_user_phone=
%env TF_VAR_user_ipaddress=

In [ ]:
#change these if you want
%env TF_VAR_redshift_user=
%env TF_VAR_redshift_password=

# Step 0 - AWS and Terraform CLI Download and Setup

## Download and Install AWS CLI

In [ ]:
%%bash
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip -qq awscliv2.zip
sudo ./aws/install

In [ ]:
!aws --version

## Download and Install Terraform

In [ ]:
TF_Ver = "0.14.8"
!wget -qq https://releases.hashicorp.com/terraform/{TF_Ver}/terraform_{TF_Ver}_linux_amd64.zip
!unzip terraform_{TF_Ver}_linux_amd64.zip && rm terraform_{TF_Ver}_linux_amd64.zip && mv terraform /usr/local/bin/

In [ ]:
!terraform --version

# Step 1 - Terraform Setup


## Directory Variables 

- You can leave them as they are or modify them as you please.
- These directorys will be in Google Drive (beginning with gdrive) will outlast the Colab Instance
- Files in directories not beginning with gdrive will disappear at the end of Colab session

In [ ]:
#python variables
AWS_CRED_DIR = "/gdrive/MyDrive/AWS/ca_dev/"
AWS_CRED_FILE = AWS_CRED_DIR + "credentials"
AWS_CONF_FILE = AWS_CRED_DIR + "config"
TF_DIR = "/gdrive/MyDrive/AWS/TF/Cadabra"
#DATA_DIR = "/data/"


In [ ]:
#shell variables using python variables
import os
os.environ['AWS_CRED_DIR'] = AWS_CRED_DIR
os.environ['AWS_CRED_FILE'] = AWS_CRED_FILE
os.environ['AWS_CONF_FILE'] = AWS_CONF_FILE
os.environ['TF_DIR'] = TF_DIR

## Check for Credentials file

https://linuxize.com/post/bash-check-if-file-exists/

In [ ]:
if os.path.isfile(AWS_CRED_FILE):
  !mkdir -p ~/.aws; cp {AWS_CRED_DIR}c* ~/.aws
else:
  !aws configure; mkdir -p {AWS_CRED_DIR}; cp  ~/.aws/c* {AWS_CRED_DIR}

In [ ]:
# This will make aws command works more consistently
import os
os.environ['AWS_PROFILE'] = "default"

In [ ]:
#Read the credentials and capture access key and secret key in python variables
f = open(AWS_CRED_FILE)
f.readline()
access_key = f.readline().split("=")[1].strip()
secret_key = f.readline().split("=")[1].strip()
f.close()

aws_region = 'us-east-1'

try:
  f = open(AWS_CONF_FILE)
  for i in range(3):
    line = f.readline()
    if line.split('=')[0].strip() == 'region':
      aws_region = line.split('=')[1].strip()
  f.close()
except:
  aws_region = 'us-east-1'

print(aws_region)

In [ ]:
!aws s3 ls

## Create the Provider File and Initialize Terraform

In [ ]:
# Create Terraform working directory
!mkdir -p $TF_DIR
%cd $TF_DIR

In [ ]:
#Define env variables for TF authentication
os.environ['AWS_ACCESS_KEY_ID'] = access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = secret_key

# Define Terraform config variables
os.environ['AWS_DEFAULT_REGION'] = aws_region

# Other variables that can be defined
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = ''
os.environ['AWS_PROFILE'] = ''
os.environ['AWS_SESSION_TOKEN'] = ''

In [ ]:
!if  [ ! -d "rsa_on_aws" ]; then git clone https://github.com/tadinve/rsa_on_aws.git; fi
%cd rsa_on_aws
!git pull https://github.com/tadinve/rsa_on_aws.git

In [ ]:
%cd src/infrastucture_install_files/terraform_files

In [ ]:
!cat provider.tf

In [ ]:
!cat  variables.tf

In [ ]:
# Clear lock from previous run it it exists
!rm .terraform.lock.hcl

In [ ]:
!terraform init

# Step 2 - Create Infra for Pipeline Kinesis(agent) -> Firehose -> Glue -> Athena


##Step 2.1 - Create Kinesis Firehose to S3

### Create Firehose Target Bucket

In [ ]:
!cat s3_firehose_bucket.tf

### Create IAM Role for Kinesis Firehose Delivery Stream

In [ ]:
!cat iam_kinesis_firehose.tf

### Create Kinesis Firehose

https://registry.terraform.io/providers/hashicorp/aws/latest/docs/resources/kinesis_firehose_delivery_stream

In [ ]:
!cat firehose.tf

## Step 2.2 - Create AWS Glue Crawler

### Create AWS Glue Role and Policy

In [ ]:
!cat iam_gluerole.tf

### Create Glue Database and Crawler

In [ ]:
!cat gluedb_crawler.tf

#Step 3 - Create Infra for Data pipeline Kinesis(agent) -> datastream -> Lambda -> DynamoDB

##Step 3.1 - Create Kinesis DataStream

In [ ]:
!cat kinesis_datastream_orders.tf

##Step 3.2 - Create Lambda Function

### Create IAM Role for Lambda Function

In [ ]:
!cat iam_lambda_role.tf

###Create Lambda Function Code

In [ ]:
%cp {TF_DIR}/rsa_on_aws/src/python_code/*.zip .

###Create Lambda Function to Process Cadabra Order

In [ ]:
!cat lambda_order_processing.tf

##Step 3.3 - Create DynamoDB Table

In [ ]:
!cat dynamodb.tf

# Step 4 - Create Infra for S3 -> Redshift -> Quicksight

## Step 4.1 Create Redshift Cluster

In [ ]:
!cat redshift_cluster.tf

## Step 4.2 Redshift Role

In [ ]:
!cat iam_redshift_role.tf

# Step 5 - Create Infra for Datastream -> Kinesis Analytics -> Datastream -> Lambda -> SNS

## Step 5.1 Create Kinesis Datastream

In [ ]:
!cat kinesis_datastream_orders.tf

## Step 5.2 Create Kinesis Application Role and Policy

In [ ]:
!cat iam_kinesis_app_role.tf

## Step 5.3 Create Kinesis Application

In [ ]:
!cat kinesis_analytics_application.tf

In [ ]:
#Copy the analytics application code to create application
%cp {TF_DIR}/rsa_on_aws/src/sql_code/*.sql .

## Step 5.4 Create Kinesis Rate Alarm Datastream for Kinesis Application Output

In [ ]:
!cat kinesis_datastream_ratealarm.tf

## Step 5.5 Create Lambda

In [ ]:
!cat lambda_surge_alert.tf

## Step 5.6 Create Lambda Role and Policy

In [ ]:
!cat iam_lambda_kinesis_sns_role.tf

## Step 5.7 Attach Trigger to Lambda

In [ ]:
!cat map_lamdba_trigger_sns_alert.tf

## Step 5.8 Create SNS Topic

In [ ]:
!cat sns_topic_subscription_sms.tf

# Step 6 - Create Infra for Kinesis Delivery Stream -> Lambda -> Elastic Search

## Step 6.1 - Create Kinesis Weblogs Delivery Stream

In [ ]:
!cat kinesis_log_deliverystream.tf

## Step 6.2 - Create Role and Policy for Kinesis Delivery Stream to write to ES

In [ ]:
!cat iam_kinesis_log_role.tf

## Step 6.3 - Create Elastic Search Domain Policy

In [ ]:
!cat iam_es_domain_policy.tf

## Step 6.4 - Create Elastic Search Domain

In [ ]:
!cat es_domain.tf

## Step 6.5 - Create Lambda Processor and Role

In [ ]:
!cat iam_lambda_weblog_deliverystream_role.tf

In [ ]:
!cat lambda_process_weblogs.tf

In [ ]:
#Copy Lambda Code for processing weblogs
%cp {TF_DIR}/rsa_on_aws/src/javascript_code/*.zip .

# Step 7 - Terraform Plan and Apply

In [ ]:
!terraform plan

In [ ]:
!terraform apply -auto-approve 

In [ ]:
!ls -alh

# Step 8 - Terraform Destroy

In [ ]:
!terraform state list

In [ ]:
!terraform destroy 